In [ ]:
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator  
from qiskit_algorithms.minimum_eigensolvers import VQE  
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp  
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import matplotlib.pyplot as plt
import datetime

Hamiltonian Equation Params

In [ ]:

# Load the provided CSV files
combined_stock_data_file = 'combined_stock_data_final.csv'
covariance_file = 'covariance (1).csv'
stock_summary_file = 'stock_summary_statistics (1).csv'

# Read the files into dataframes
combined_stock_data = pd.read_csv(combined_stock_data_file)
covariance_data = pd.read_csv(covariance_file)
stock_summary_data = pd.read_csv(stock_summary_file)

# Display the contents of the files to get an understanding of the data
combined_stock_data.head(), covariance_data.head(), stock_summary_data.head()


## Hamiltonian constructor

we're using a built in function called Portfolio Optiizer that acts as the constructor

## Suggested Risk Aversion Levels:
### Low Risk Aversion (10-30%):
 Suitable if you’re focused on maximizing returns and are willing to tolerate some volatility. This range is more aggressive and is typical for growth-oriented investors.
### Moderate Risk Aversion (30-50%): 
This strikes a balance between growth and risk. It’s suitable if you want solid returns but are also mindful of protecting your portfolio during downturns. For tech stocks, this could be an ideal range.
### High Risk Aversion (50-70%): 
Suitable for very conservative investors who prioritize minimizing losses over maximizing gains. This is often used in portfolios with bonds or defensive stocks, but for safe tech stocks, it could be too conservative.

In [ ]:
# Extract relevant data for constructing the Hamiltonian
mean_returns = stock_summary_data['Mean Return'].values
cov_matrix = covariance_data.iloc[:, 1:].values  # Exclude first column which has stock names

# Step 1: Define risk aversion and budget for portfolio optimization
risk_aversion = 0.5
budget = 100000  # Budget of 100,000 USD

# Step 2: Initialize PortfolioOptimization with budget
portfolio_optimization = PortfolioOptimization(
    expected_returns=mean_returns,
    covariances=cov_matrix,
    risk_factor=risk_aversion,
    budget=budget  # Budget constraint
)

# Step 3: Manually construct the Hamiltonian using SparsePauliOp
from qiskit.quantum_info import Pauli

num_assets = len(mean_returns)
terms = []
coeffs = []

# Loop through assets to construct the Hamiltonian
for i in range(num_assets):
    # Risk part: Covariance terms
    for j in range(i + 1, num_assets):
        pauli_label = 'Z' * i + 'I' * (num_assets - i - 1)
        pauli = Pauli(pauli_label)
        coeff = cov_matrix[i, j]
        terms.append(pauli)
        coeffs.append(coeff)
    
    # Return part: Mean return term
    pauli_label = 'Z' * i + 'I' * (num_assets - i - 1)
    pauli = Pauli(pauli_label)
    coeff = -mean_returns[i]
    terms.append(pauli)
    coeffs.append(coeff)

# Convert into SparsePauliOp
hamiltonian = SparsePauliOp.from_list([(p.to_label(), c) for p, c in zip(terms, coeffs)])


Ansatz is the curcuit used in a VQE. We can use a pre-built one for now. TwoLocal is used in VQEs

We'll use the built in VQE algorithm to create the output. All built in functions used were created by IBM engineers

In [ ]:
# Step 4: Define a simple ansatz (e.g., TwoLocal) for the VQE
ansatz = TwoLocal(num_qubits=num_assets, reps=2, rotation_blocks='ry', entanglement_blocks='cz')

# Step 5: Set up VQE with the Estimator, ansatz, and COBYLA optimizer
estimator = Estimator()
cobayla = COBYLA(maxiter=100)
vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=cobayla)

# Step 6: Run VQE to compute the minimum eigenvalue of the Hamiltonian
result = vqe.compute_minimum_eigenvalue(hamiltonian)

# Step 7: Output the results (minimum eigenvalue and optimal portfolio configuration)
print("Minimum Eigenvalue (Risk-Return Tradeoff):", result.eigenvalue.real)
# print("Optimal Portfolio Parameters (Qubit Rotations):", result.optimal_parameters)

def print_readable_optimal_params(optimal_params):
    print("\nOptimal Portfolio Parameters (Qubit Rotations):")
    
    for param, value in optimal_params.items():
        # The 'param' is a ParameterVectorElement object, which has a label 'θ[i]'
        print(f"{param}: {value:.4f}")

# Run VQE to compute the minimum eigenvalue of the Hamiltonian
result = vqe.compute_minimum_eigenvalue(hamiltonian)

# Step 7: Output the results (minimum eigenvalue and optimal portfolio configuration)
print("Minimum Eigenvalue (Risk-Return Tradeoff):", result.eigenvalue.real)
print_readable_optimal_params(result.optimal_parameters)

Readable output that selects the stocks to add to our portfolio

In [ ]:
def select_optimal_portfolio(optimal_params, asset_names, budget, threshold=np.pi/2):
    """
    Convert optimal parameters (rotation angles) into binary decisions for portfolio selection.
    Assign a portion of the total budget based on the parameter values.
    
    Args we used:
    - optimal_params: Dictionary of optimal rotation angles from the VQE result.
    - asset_names: List of asset names by ticker.
    - budget: The total amount of money available to invest.
    - threshold: The threshold for deciding whether to include a stock in the portfolio.
    
    Returns:
    - A list of selected assets.
    """
    print("\nOptimal Portfolio Selection:")
    
    selected_assets = []
    allocations = {}
    
    num_assets = len(asset_names)
    param_values = list(optimal_params.values())  # Extract the parameter values

    # Group parameters per asset (assuming an even distribution of parameters across assets)
    grouped_params = np.array_split(param_values, num_assets)
    
    # Calculate the aggregated absolute parameter for each asset
    aggregated_params = [abs(np.mean(group)) for group in grouped_params]

    # Filter the selected assets and sum of aggregated parameters for normalization
    total_param_sum = sum(aggregated_params)  # Sum of aggregated parameters to compute weight distribution
    
    for i, asset_name in enumerate(asset_names):
        # Aggregate parameters for each asset
        aggregated_param = aggregated_params[i]
        
        # Decide if we include the stock based on the threshold
        if aggregated_param > threshold:
            decision = "Include"
            selected_assets.append(asset_name)
            
            # Calculate the percentage of the budget to allocate to this stock
            weight = aggregated_param / total_param_sum
            money_allocated = budget * weight
            allocations[asset_name] = {
                'Percentage': weight * 100,
                'Monetary': money_allocated
            }
        else:
            decision = "Do not include"
        
        print(f"Stock {asset_name}: {decision} (Aggregated Rotation = {aggregated_param:.4f})")
    
    return selected_assets, allocations

asset_names = ['AAPL', 'AMZN', 'GOOGL', 'FB', 'NVDA', 'ADBE', 'MSFT', 'AMD', 'INTC', 'TSLA', 'IBM', 'ORCL', 'JPM', 'CRM', 'MS', 'NOW', 'PLTR', 'LMT', 'DIS', 'WFC']  # Example stock names
budget = 100000  # Example budget of $100,000

result = vqe.compute_minimum_eigenvalue(hamiltonian)

# Select optimal portfolio based on threshold and calculate allocations
selected_assets, allocations = select_optimal_portfolio(result.optimal_parameters, asset_names, budget)

# Print final selected assets and their allocations
print("\nSelected Assets for Optimal Portfolio and Budget Allocations:")
for asset, alloc in allocations.items():
    print(f"{asset}: {alloc['Percentage']:.2f}% of budget, ${alloc['Monetary']:.2f}")
